<h1>Kalman Filter Experiments</h1>

In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from math import cos, sin, tan, atan2

output_notebook()

<h2>Creating the data</h2>

First of all a dataset has to be created. Below a dataset was constructed where a vehicle drives a simple track. First a vehicle model has to be implemented and in this case a Bicycle Model is used.

In [2]:
normalise_angle = lambda angle: atan2(sin(angle), cos(angle))

class KinematicBicycleModel:
    """
    Summary
    -------
    This class implements the 2D Kinematic Bicycle Model for vehicle dynamics

    Attributes
    ----------
    dt (float) : discrete time period [s]
    wheelbase (float) : vehicle's wheelbase [m]
    max_steer (float) : vehicle's steering limits [rad]

    Methods
    -------
    __init__(wheelbase: float, max_steer: float, delta_time: float=0.05)
        initialises the class

    update(x, y, yaw, velocity, acceleration, steering_angle)
        updates the vehicle's state using the kinematic bicycle model
    """
    def __init__(self, wheelbase: float, max_steer: float, delta_time: float=0.05):

        self.delta_time = delta_time
        self.wheelbase = wheelbase
        self.max_steer = max_steer


    def update(self, x: float, y: float, yaw: float, velocity: float, acceleration: float, steering_angle: float) -> tuple[float, ...]:
        """
        Summary
        -------
        Updates the vehicle's state using the kinematic bicycle model

        Parameters
        ----------
        x (int) : vehicle's x-coordinate [m]
        y (int) : vehicle's y-coordinate [m]
        yaw (int) : vehicle's heading [rad]
        velocity (int) : vehicle's velocity in the x-axis [m/s]
        acceleration (int) : vehicle's accleration [m/s^2]
        steering_angle (int) : vehicle's steering angle [rad]

        Returns
        -------
        new_x (int) : vehicle's x-coordinate [m]
        new_y (int) : vehicle's y-coordinate [m]
        new_yaw (int) : vehicle's heading [rad]
        new_velocity (int) : vehicle's velocity in the x-axis [m/s]
        steering_angle (int) : vehicle's steering angle [rad]
        angular_velocity (int) : vehicle's angular velocity [rad/s]
        """
        # Compute the local velocity in the x-axis
        new_velocity = velocity + self.delta_time * acceleration

        # Limit steering angle to physical vehicle limits
        steering_angle = -self.max_steer if steering_angle < -self.max_steer else self.max_steer if steering_angle > self.max_steer else steering_angle

        # Compute the angular velocity
        angular_velocity = new_velocity*tan(steering_angle) / self.wheelbase

        # Compute the final state using the discrete time model
        new_x   = x + velocity*cos(yaw)*self.delta_time
        new_y   = y + velocity*sin(yaw)*self.delta_time
        new_yaw = normalise_angle(yaw + angular_velocity*self.delta_time)

        return new_x, new_y, new_yaw, new_velocity, steering_angle, angular_velocity

After the Bicycle Model is implemented, an arbitrary vehicle can be modeled. In this case the model of a Volvo S60 was chosen.

In [3]:
timeStep = 0.05
dt = timeStep
modelIterations = 200

model = KinematicBicycleModel(2.872, 0.64, timeStep) # the parameters of a Volvo S60, the best car
positionGroundTruthX = [0]
positionGroundTruthY = [0]
timeAtPosition = np.arange(0, timeStep * modelIterations, timeStep)

x = 0
y = 0
yaw = 0
velocity = 0
velocity_history = [0]
acceleration = np.zeros(modelIterations)
steering_angle = np.zeros(modelIterations)

noiseAcceleration = np.random.normal(0.05, 0.05, modelIterations)
noiseSteeringStart = np.random.normal(0.01, 0.01, modelIterations // 4)
noiseSteeringMiddle = np.random.normal(-0.02, 0.01, modelIterations // 2)
noiseSteeringEnd = np.random.normal(0.01, 0.005, modelIterations - modelIterations // 4)

for i in range(1, modelIterations):
    acceleration[i] = acceleration[i-1] + noiseAcceleration[i]
    if i < modelIterations // 4:
        steering_angle[i] = steering_angle[i-1] + noiseSteeringStart[i]
    elif i > (modelIterations // 4) - 1 and i < modelIterations // 2:
        steering_angle[i] = steering_angle[i-1] + noiseSteeringMiddle[i-modelIterations // 4]
    else:
        steering_angle[i] = steering_angle[i-1] + noiseSteeringEnd[i-(modelIterations - modelIterations // 4)]

for i in range(0, modelIterations - 1):
    x, y, yaw, velocity, _, _ = model.update(x, y, yaw, velocity, acceleration[i], steering_angle[i])
    positionGroundTruthX.append(x)
    positionGroundTruthY.append(y)
    velocity_history.append(velocity)

The model calculated a trajectory from the given acceleration and steering inputs. These inputs are generated in a way that the vehicle accelerates the whole time and gets faster in every time step. The steering angle is chosen to change between going fully left and right in a way which is not completely smooth to simulate road bumbs or irregularities on the surface.<br>
Lets look at the trajectory.

In [4]:
# x, y position map
p = figure(title="Ground Truth Data", x_axis_label='x Position', y_axis_label='y Position')
p.toolbar_location = None
p.line(positionGroundTruthX, positionGroundTruthY, line_width=2, color='red')
show(p)

The trajectory shows a smooth movement which overlaps on some places. It also can be seen that when driving with high velocities the model is not as smooth anymore because the distance travelled in the short time frame is not negligible.<br>
Lets look at the individual values and how they develop over time.

In [5]:
# acceleration over time
p = figure(title="Acceleration in m/s2", x_axis_label='Time in seconds', y_axis_label='acceleration')
p.toolbar_location = None
p.line(timeAtPosition, acceleration, line_width=2, color='red')
show(p)

# velocity over time
p = figure(title="velocity in m/s", x_axis_label='Time in seconds', y_axis_label='velocity')
p.toolbar_location = None
p.line(timeAtPosition, velocity_history, line_width=2, color='red')
show(p)

# steering angle over time
p = figure(title="Steering angle in rad", x_axis_label='Time in seconds', y_axis_label='angle')
p.toolbar_location = None
p.line(timeAtPosition, steering_angle, line_width=2, color='red')
show(p)

All of the values work as expected and could be taken from a real life scenario. <br>
In order to test the Kalman Filter, sensor data has to be generated. Since there is no access to the simulation environment right now, the sensor data will be the ground truth data multiplied with some gaussian noise. Since the random seed is not set, the data will be different every time this cell is run just like the track itself.

In [6]:
# IMU position data
noiseX = np.random.normal(1, 0.02, modelIterations)
noiseY = np.random.normal(1, 0.02, modelIterations)
positionIMUX = np.multiply(positionGroundTruthX, noiseX)
positionIMUY = np.multiply(positionGroundTruthY, noiseY)

# GNSS position data
noiseX = np.random.normal(1, 0.02, modelIterations)
noiseY = np.random.normal(1, 0.02, modelIterations)
positionGNSSX = np.multiply(positionGroundTruthX, noiseX)
positionGNSSY = np.multiply(positionGroundTruthY, noiseY)


p = figure(title="Comparison of Ground Truth to IMU and GNSS data", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')

show(p)

It can be seen that the sensor data is always near the track but sometimes has some significant outliers. This representation is not perfect as each sensor has its individual flaws but it is a good starting point.

<h2>The Kalman Filter</h2>

The Kalman Filter will be implemented as described in [An Introduction to the Kalman Filter](http://www.cs.unc.edu/~welch/media/pdf/kalman_intro.pdf). In this first implementation the most simple form will be implemented.

In [7]:
class KalmanFilter:

    def __init__(self, dt, A, H, Q, R, P):
        self.A = A
        self.H = H
        self.Q = Q
        self.R = R
        self.P0 = P
        self.m = np.shape(H)[0]
        self.n = np.shape(A)[0]
        self.dt = dt
        self.initialized = False
        self.I = np.identity(self.n)
        self.x_hat = np.zeros(self.n)
        self.x_hat_new = np.zeros(self.n)

    def init(self, t0=None, x0=None):
        if x0 is not None:
            self.x_hat = x0
        self.P = self.P0
        self.t0 = 0
        if t0 is not None:
            self.t0 = t0
        self.t = self.t0
        self.initialized = True

    def update(self, y, dt=None, A=None):
        if A is not None:
            self.A = A
        if dt is not None:
            self.dt = dt

        if not self.initialized:
            print("Filter not initialized")

        self.x_hat_new = self.A @ self.x_hat
        self.P = self.A @ self.P @ np.transpose(self.A) + self.Q
        self.K = self.P @ np.transpose(self.H) @ np.linalg.inv(self.H @ self.P @ np.transpose(self.H) + self.R)
        self.x_hat_new = self.x_hat_new + self.K @ (y - self.H @ self.x_hat_new)
        self.P = (self.I - self.K @ self.H) @ self.P
        self.x_hat = self.x_hat_new

        self.t = self.t + self.dt

def runFilter(A, H, Q, R, P):
    # list of all values to plot
    x_hatXValues = []
    x_hatYValues = []

    filter = KalmanFilter(dt, A, H, Q, R, P)
    filter.init()

    for xIMU, yIMU, xGNSS, yGNSS in zip(positionIMUX, positionIMUY, positionGNSSX, positionGNSSY):
        y = np.array([xIMU, yIMU, xGNSS, yGNSS])
        filter.update(y)
        x_hatXValues.append(filter.x_hat[0])
        x_hatYValues.append(filter.x_hat[1])

    return x_hatXValues, x_hatYValues

Now an instance of the Kalman Filter can be created. The parameters will be explained in a later chapter. <br>
Lets look at an example.

In [8]:
# Model
A = np.array([[0.01, 0], [0, 0.01]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[50, 0], [0, 50]])
R = np.array([[0.001, 0, 0, 0], [0, 0.001, 0, 0], [0, 0, 0.001, 0], [0, 0, 0, 0.001]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

The Kalman Filter now has access to the sensor data visualized above and tries to estimate the real state. Lets take a look at the visualisation of the output of the Filter.

In [9]:
p = figure(title="Output of the Kalman Filter", x_axis_label='x Position', y_axis_label='y Position')

# filter output
p.line(x_hatXValues, x_hatYValues, line_width=2, color='orange')

show(p)

The result looks a lot like the pre-defined track but lets compare the Filter to the Ground Truth.

In [10]:
p = figure(title="Comparison of Filter and Ground Truth", x_axis_label='x Position', y_axis_label='y Position')

# filter output
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')
# ground truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label="Ground Truth", line_width=2, color='red')

show(p)

The tracks overlap really well, with some error on some positions. If we take a look at the sensor data we can see why these errors occur.

In [11]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

The figure shows that the filter follows the path based of the sensor data provided. The errors produced by the filter are in points in which the sensor data is way of and therefore it is very difficult for the filter to adjust. Why this is the case will be explained in the next chapter.

<h3>Understanding the Parameters</h3>

<h4>Tweaking the system dynamics matrix A</h4>

The system dynamics matrix A is used to describe how the observed process changes over time.<br>
First lets look at a low value of A.

In [12]:
# Model
A = np.array([[0, 0], [0, 0]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[1, 0], [0, 1]])
R = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

If the matrix A is discarded, like here with a value of 0, the filter does no initial prediction before getting the measurements from the sensors. This means that the process will only be influenced by the mesaurements and therefore the value of R becomes very important.<br>
Lets look at a high value of A.

In [13]:
# Model
A = np.array([[3, 0], [0, 3]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[1, 0], [0, 1]])
R = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

If A has a high value, the filter tries to predict the state without the sensor data and then adjusts with the measurements. But because this process is very instable, since the dynamics change in every step, the performance of the filter decreases with a high A value because we overshoot the track. This is due to the filter going forward a lot and then adjusting with the measurements.

<h4>The output matrix H</h4>

The matrix H translates the state vector to the measurement vector. That means it tells the filter how the measurements correlate to the states. Therefore it should not be tweaked but rather calculated. Since in this case the measurements of the position directly relate to the position state, the matrix looks very simple.

<h4>Tweaking process noise covariance matrix Q</h4>

The process noise covariance matrix Q describes how instable the process itself is. This can be best understood if the process is constant not like in this case. But lets take a look at the edge cases anyway starting with a low value of Q.

In [14]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[0.001, 0], [0, 0.001]])
R = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

If the value of Q is low, the filter believes in a very stable system and therefore does not adjust the prediction as much. Therefore it can be seen that the result does not increase its value fast enough.

In [15]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[100, 0], [0, 100]])
R = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

If the value of Q is high, the filter believes that the system is very unstable and that the results vary a lot in the different states. Therefore it provides better results in this use case.

<h4>Tweaking the meaurement noise covariance matrix R</h4>

The measurement noise covariance matrix describes how accurate the sensors are and how much we trust their measurements. Therefore low values in R make the filter adjust more to the measurements and high values in R make the filter trust its own prediction more.<br>
First lets look at a low value of R.

In [16]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[1, 0], [0, 1]])
R = np.array([[0.001, 0, 0, 0], [0, 0.001, 0, 0], [0, 0, 0.001, 0], [0, 0, 0, 0.001]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

With a low value of R the system basically becomes a mean of the sensor values as can be seen in the figure.<br>
And now lets look at a high value of R.

In [17]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])

# Covariance matrices
Q = np.array([[1, 0], [0, 1]])
R = np.array([[1000, 100, 100, 100], [100, 1000, 100, 100], [100, 100, 1000, 100], [100, 100, 1000, 1000]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P)

p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

As one can see in the figure, the filter does not really move at all, but the direction is mostly right. This is because, with a R value this high, the filter is very dependent on the system dynamics matrix A and this value is not that useful to us, since in this application we do not know the system dynamics and therefore want to depent a lot on the measurements.

<h4>Changing the initial state of the estimate error covariance matrix P</h4>

The error covaricance matrix P describes the estimated error to the ground truth and therefore changes every state. Its initial value does not matter as much as it converges over time.

<h3>The control input u</h3>

At the moment the filter struggles to predict the state because the system dynamics are constantly changing and therefore a control input u is added. The input vector u in combination with a translation matrix or model B gives the filter additional information about the system. Usually in these kind of applications a vehicle model would be used in order to tell the filter what the expected change in state is for the given input vector u.<br>
The vehicle model is initially used in the first step to generate the data. If the same model was used the Kalman Filter would generate exceptionally good results and the sensor data could basically be discarded.<br>
First the Kalman Filter has to be adjusted to accept an input u.

In [18]:
class KalmanFilter:

    def __init__(self, dt, A, H, Q, R, P, B):
        self.A = A
        self.H = H
        self.Q = Q
        self.R = R
        self.P0 = P
        self.B = B
        self.m = np.shape(H)[0]
        self.n = np.shape(A)[0]
        self.dt = dt
        self.initialized = False
        self.I = np.identity(self.n)
        self.x_hat = np.zeros(self.n)
        self.x_hat_new = np.zeros(self.n)
        self.x_hat_model_new = np.zeros(self.n)
        self.velocity = 0
        self.yaw = 0

    def init(self, t0=None, x0=None):
        if x0 is not None:
            self.x_hat = x0
        self.P = self.P0
        self.t0 = 0
        if t0 is not None:
            self.t0 = t0
        self.t = self.t0
        self.initialized = True

    def update(self, y, u, dt=None, A=None):
        if A is not None:
            self.A = A
        if dt is not None:
            self.dt = dt

        if not self.initialized:
            print("Filter not initialized")

        # model estimation
        modelX, modelY, self.yaw, self.velocity, _, _ = self.B.update(self.x_hat[0], self.x_hat[1], self.yaw, self.velocity, u[0], u[1])
        self.x_hat_model_new = np.array([modelX, modelY])

        self.x_hat_new = (self.A @ self.x_hat) + self.x_hat_model_new - self.x_hat
        self.P = self.A @ self.P @ np.transpose(self.A) + self.Q
        self.K = self.P @ np.transpose(self.H) @ np.linalg.inv(self.H @ self.P @ np.transpose(self.H) + self.R)
        self.x_hat_new = self.x_hat_new + self.K @ (y - self.H @ self.x_hat_new)
        self.P = (self.I - self.K @ self.H) @ self.P
        self.x_hat = self.x_hat_new

        self.t = self.t + self.dt

def runFilter(A, H, Q, R, P, B):
    # list of all values to plot
    x_hatXValues = []
    x_hatYValues = []

    filter = KalmanFilter(dt, A, H, Q, R, P, B)
    filter.init()

    for xIMU, yIMU, xGNSS, yGNSS, accelerationAtTimeStep, steeringAngleAtTimeStep in zip(positionIMUX, positionIMUY, positionGNSSX, positionGNSSY, acceleration, steering_angle):
        y = np.array([xIMU, yIMU, xGNSS, yGNSS])
        u = np.array([accelerationAtTimeStep, steeringAngleAtTimeStep])
        filter.update(y, u)
        x_hatXValues.append(filter.x_hat[0])
        x_hatYValues.append(filter.x_hat[1])

    return x_hatXValues, x_hatYValues

After the adjustments were made, an input vector u with acceleration and steering values now can be added into the filter. This has a lot more use cases in the real world especially when looking at vehicles. Lets generate a vehicle model and look at the results.

In [19]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])
B = KinematicBicycleModel(2.872, 0.64, timeStep)

# Covariance matrices
Q = np.array([[0.001, 0], [0, 0.001]])
R = np.array([[1000, 0, 0, 0], [0, 1000, 0, 0], [0, 0, 1000, 0], [0, 0, 0, 1000]])
P = np.array([[0.001, 0], [0, 0.001]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P, B)

In this case, the same vehicle model as in the generation of the dataset is used. Therefore the model could cheat and discard all of the sensor measurements because it could just replicate the track with the control inputs.<br>
Lets visualize the result when the sensor data is basically erased from the filter.

In [20]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

The red line vanishes under the yellow line because the Kalman Filter estimates the track perfectly. This is no surprise because as explained above the filter now uses the perfect vehicle model to estimate the state and the parameters were tweaked in a way so that the sensor data is basically ignored by the filter, by choosing a very low Q and P and a very high R. <br>
But of course in the real world, the perfect model does not exist and the control input u could also never be estimated this perfectly. There are always some errors in the system or influences which arise from outside the system. <br>
Therefore lets look at an example which is more realistic in which we approximate the model and also take our sensor values into account.

In [21]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])
B = KinematicBicycleModel(2.9, 0.6, timeStep)

# Covariance matrices
Q = np.array([[1, 0], [0, 1]])
R = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
P = np.array([[1, 0], [0, 1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P, B)

This model now uses a different length and a different maximum steering range. The second one does not matter as much but just to show that it could also be approximated. For the covariance matrices everything was set to 1 so the model evaluates the model just as much as the sensor data.<br>
Lets plot the result.

In [22]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(positionIMUX, positionIMUY, legend_label='IMU', line_width=2, color='blue')
p.line(positionGNSSX, positionGNSSY, legend_label='GNSS', line_width=2, color='green')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

The result looks quite good. There are some outliers where the sensor data is way off, but other than that, the result is very close to the ground truth. <br>
Lets remove some clutter by discarding the sensor data in the visualization.

In [23]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

The filter and the ground truth look very alike. Now what happens if the filter weights either the model or the sensor data more. Lets implement a filter which evaluates the model more.

<h4>Model heavy evaluation</h4>

This set of parameters makes the filter trust the model more than the sensor data.

In [24]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])
B = KinematicBicycleModel(2.9, 0.6, timeStep)

# Covariance matrices
Q = np.array([[0.1, 0], [0, 0.1]])
R = np.array([[10, 0, 0, 0], [0, 10, 0, 0], [0, 0, 10, 0], [0, 0, 0, 10]])
P = np.array([[0.1, 0], [0, 0.1]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P, B)

Lets visualize it and look at the result.

In [25]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

It can be seen that the generated track by the filter is very smooth as it is heavily influenced by the vehicles dynamics. But it can also be seen that the small error in the model accumulates over time and therefore the position drifts away. Since the sensor data is not trusted as much it is not able to correct the output.

<h4>Sensor heavy evaluation</h4>

This set of parameters makes the filter trust the sensor data more than the model.

In [26]:
# Model
A = np.array([[1, 0], [0, 1]])
H = np.array([[1, 0], [0, 1], [1, 0], [0, 1]])
B = KinematicBicycleModel(2.9, 0.6, timeStep)

# Covariance matrices
Q = np.array([[10, 0], [0, 10]])
R = np.array([[0.1, 0, 0, 0], [0, 0.1, 0, 0], [0, 0, 0.1, 0], [0, 0, 0, 0.1]])
P = np.array([[10, 0], [0, 10]])

x_hatXValues, x_hatYValues = runFilter(A, H, Q, R, P, B)

Lets visualize it and look at the result.

In [27]:
p = figure(title="Comparison of Ground Truth, IMU, GNSS and the filter output", x_axis_label='x Position', y_axis_label='y Position')

# ground Truth
p.line(positionGroundTruthX, positionGroundTruthY, legend_label='Ground Truth', line_width=2, color='red')
p.line(x_hatXValues, x_hatYValues, line_width=2, legend_label="Kalman Filter", color='orange')

show(p)

The result is much closer to the actual track but the outliers impact the filter more than before. Therefore the result is not as smooth anymore and it can be seen that it was not derived from the vehicle dynamics.

<h3>Conclusion</h3>

In the end a balance has to be found between the model and the sensor data. This always depends on the use case and cannot be generalized. In this case where the model is known it is a very obvious choice but in the real world the model suffers from a lof of influences from outside the system. But one cannot trust the sensors too much since they also have their problems. Therefore, the only solution is testing thoroughly and adjusting the filter until it works the best. <br>
In this short overview even though the parameters were explained and tested in the edge cases they can of course be fine-tuned a lot.